In [63]:
import geopandas as gpd
import pandas as pd
import folium
import pyproj
import shapely
import matplotlib.pyplot as plt

In [67]:
# 프로젝션 정의
xy4326 = pyproj.CRS('epsg:4326')  
xy5179 = pyproj.CRS('epsg:5179')  
xy5186 = pyproj.CRS('epsg:5186')

# Transformer 생성
transformer = pyproj.Transformer.from_crs(xy5186, xy4326)
# transformer_86 = pyproj.Transformer.from_crs(xy5186, xy5186)


In [65]:
processing_data = pd.read_csv('data/test_data.csv')

In [ ]:
transformer.transform(processing_data.loc[0,['x','y']].values[0], processing_data.loc[0,['x','y']].values[1])

In [118]:
processing_final = processing_data.assign(
    latitude = lambda df: df.apply(lambda row: transformer.transform(row.x, row.y)[0], axis=1),
    logitude = lambda df: df.apply(lambda row: transformer.transform(row.x, row.y)[1], axis=1),
)
processing_geo = gpd.GeoDataFrame(processing_final.assign(geometry = lambda df:df.apply(lambda row:shapely.geometry.Point((row.x, row.y)), axis=1)))
sgg_select = processing_geo.filter(regex='시도|시군구').drop_duplicates()\
.apply(lambda row: f"{row['시도']} {row['시군구'].split()[-1]}" if "부천" not in row['시군구'] 
       else f"{row['시도']} {' '.join(row['시군구'].split()[1:])}" , axis=1).tolist()

In [119]:
seoul_geo = gpd.read_file('data/seoul_geo/LARD_ADM_SECT_SGG_11_202405.shx', encoding = 'cp949').query("SGG_NM in @sgg_select")
gyeonggi_geo = gpd.read_file('data/gyeonggi_geo/LARD_ADM_SECT_SGG_41_202405.shx', encoding = 'cp949').query("SGG_NM in @sgg_select")
incheon_geo = gpd.read_file('data/incheon_geo/LARD_ADM_SECT_SGG_28_202405.shx', encoding = 'cp949').query("SGG_NM in @sgg_select")

In [ ]:
sgg_select

In [ ]:
fig, ax = plt.subplots()
seoul_geo.plot(ax=ax)
gyeonggi_geo.plot(ax = ax)
incheon_geo.plot(ax = ax)
processing_geo.plot(ax=ax, color = 'r')